In [9]:
import torch
print(f'Using torch version {torch.__version__}')
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name()
    device_memory = torch.cuda.get_device_properties(0).total_memory
    device_memory /= 1e9
    print(f'Found GPU: {torch.cuda.get_device_name()}, with {device_memory} GB of memory')

batch_size = int(device_memory/3)
print(batch_size)

Using torch version 1.13.1+cu116
Found GPU: NVIDIA GeForce RTX 3060, with 12.8843776 GB of memory
4


In [2]:
import os
os.chdir('../')

# from promptehr import load_demo_data

# load pytrial demodata, supported by PyTrial package to load the demo EHR data
from pytrial.data.demo_data import load_mimic_ehr_sequence
from pytrial.tasks.trial_simulation.data import SequencePatient

# see the input format
data_dir = 'C:\\Users\\noah\\cse517_project_code\\data_conversion\\datasets\\train'
# val_dir = 'C:\\Users\\noah\\cse517_project_code\\data_conversion\\datasets\\val'
demo = load_mimic_ehr_sequence(input_dir=data_dir, n_sample=15000)
# val_data = load_mimic_ehr_sequence(input_dir=val_dir)

# build sequence dataset
seqdata = SequencePatient(data={'v':demo['visit'], 'y':demo['mortality'], 'x':demo['feature'],},
    metadata={
        'visit':{'mode':'dense'},
        'label':{'mode':'tensor'}, 
        'voc':demo['voc'],
        'max_visit':20,
        }
    )

# val_seqdata = SequencePatient(data={'v':val_data['visit'], 'y':val_data['mortality'], 'x':val_data['feature'],},
#     metadata={
#         'visit':{'mode':'dense'},
#         'label':{'mode':'tensor'}, 
#         'voc':val_data['voc'],
#         'max_visit':20,
#         }
#     )

print('visit', demo['visit'][0]) # a list of visit events
print('mortality', demo['mortality'][0]) # array of labels
print('feature', demo['feature'][0]) # array of patient baseline features
print('voc', demo['voc']) # dict of dicts containing the mapping from index to the original event names
print('order', demo['order']) # a list of three types of code
print('n_num_feature', demo['n_num_feature']) # int: a number of patient's numerical features
print('cat_cardinalities', demo['cat_cardinalities']) # list: a list of cardinalities of patient's categorical features

c:\Users\noah\miniconda3\envs\cse517\lib\site-packages\pytrial\data\patient_data.py:95: UserWarning: No metadata provided. Metadata will be automatically detected from your data. This process may not be accurate. We recommend writing metadata to ensure correct data handling.
  warnings.warn('No metadata provided. Metadata will be automatically '


In [3]:
from promptehr import PromptEHR

# fit the model
model = PromptEHR(
    code_type=demo['order'],
    n_num_feature=demo['n_num_feature'],
    cat_cardinalities=demo['cat_cardinalities'],
    num_worker=0,
    eval_step=20000,
    epoch=1,
    batch_size=4,
    # device=[1,2],
    output_dir='C:\\Users\\noah\\cse517_project_code\\promptEHR_logs'
)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'DataTokenizer'.


In [4]:
model.fit(
    train_data=seqdata,
    val_data=seqdata,
)

c:\Users\noah\miniconda3\envs\cse517\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 15000
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 3750
  Number of trainable parameters = 145501440


  0%|          | 0/3750 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (556 > 512). Running this sequence through the model will result in indexing errors


{'loss': 5.2681, 'learning_rate': 3.900709219858156e-05, 'epoch': 0.27}
evaluation for code diag.
***** Running Evaluation *****
  Num examples = 15000
  Batch size = 16


  0%|          | 0/938 [00:00<?, ?it/s]

KeyboardInterrupt: 